## 로컬 환경에서 PDF 검색하기 2단계 step1
- PDF 문서를 로드하고 한국어 임베딩 모델을 사용하여 임베딩 데이터 생성
- 임베딩 데이터를 csv 파일로 만들어 저장하기 
- 저장한 csv 데이터를 읽어서 FAISS 인덱스 생성하기
- 생성한 FAISS 인덱스를 검색하기 

### - 사용한 임베딩 모델 jhgan/ko-sroberta-multitask


In [6]:
# 라이브러리 설치 (필요한 경우 실행)
%pip install -U langchain langchain_core sentence-transformers faiss-cpu pymupdf

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 필요한 라이브러리 임포트
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings  
from sentence_transformers import SentenceTransformer

# Step 1: 문서 로드
pdf_file_path = "data/"
pdf_file_name = "AI기반_인파분석플랫폼구축_제안서"

loader = PyMuPDFLoader(pdf_file_path + pdf_file_name + ".pdf") 
docs = loader.load()

# Step 2: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [3]:
# Step 3: SentenceTransformer 모델을 LangChain의 Embeddings 클래스로 감싸기
class KoSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        """문서 리스트를 벡터로 변환"""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        """검색 쿼리를 벡터로 변환"""
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]


# Step 4: 모델 로드 및 FAISS 인덱스 생성
embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")
# faiss_index = FAISS.from_documents(split_documents, embedding_model)

In [7]:
import csv
import pandas as pd
import os

# Step 5: 문서 임베딩 및 CSV 저장
# 원본 문서와 임베딩 데이터를 CSV 에 함께 저장함
# 원본 문서를 저장하는 이유 : 검색 결과를 보여줘야 하기 때문. 
def save_embeddings_to_csv(documents, embedding_model, filename=pdf_file_name+".csv", file_path="./csv/"):
    # 경로가 존재하지 않은 경우 디렉토리 생성
    os.makedirs(file_path, exist_ok=True)
    full_path = os.path.join(file_path, filename)

    # 문서 임베딩 수행
    embeddings = embedding_model.embed_documents([doc.page_content for doc in documents])
    
    # CSV 저장
    with open(full_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["document", "embedding"])
        
        for doc, embedding in zip(documents, embeddings):
            writer.writerow([doc.page_content, embedding])
    
    print(f"임베딩 데이터가 {full_path} 파일에 저장되었습니다.")
    return full_path

# 함수 실행
documents = split_documents  # FAISS에 넣은 문서 리스트 사용
full_path = save_embeddings_to_csv(documents, embedding_model)

임베딩 데이터가 ./csv/AI기반_인파분석플랫폼구축_제안서.csv 파일에 저장되었습니다.


In [11]:
import faiss
import numpy as np
import pandas as pd


# CSV 파일 불러오기 
def load_embeddings_from_csv(filepath):
    df = pd.read_csv(filepath)
    df["embedding"] = df["embedding"].apply(lambda x: np.fromstring(x[1:-1], sep=','))  # 문자열을 numpy 배열로 변환
    return df

# FAISS 인덱스 생성
def create_faiss_index(embedding_dim, df):
    index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 인덱스
    embeddings = np.vstack(df["embedding"].values).astype("float32")
    index.add(embeddings)  
    return index, df

# CSV에서 데이터 불러오기
df_embeddings = load_embeddings_from_csv(full_path)

# FAISS 인덱스 생성
embedding_dim = len(df_embeddings["embedding"].iloc[0])  # 벡터 차원 수 확인
faiss_index, df_embeddings = create_faiss_index(embedding_dim, df_embeddings)

print("FAISS 인덱스가 성공적으로 생성되었습니다!")

FAISS 인덱스가 성공적으로 생성되었습니다!


In [14]:
# 생성한 FAISS 인덱스로 검색하기 
def search_faiss_index(query_embedding, index, df, k=5):
    query_vector = np.array(query_embedding).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_vector, k)
    
    results = []
    for i in range(k):
        idx = indices[0][i]
        results.append((df.iloc[idx]["document"], distances[0][i]))  # (문서 내용, 거리) 반환
    return results

# 예제 쿼리 실행
query_text = "이 문서의 주제가 뭐야?"  # 검색할 문장
query_embedding = embedding_model.embed_query(query_text)  # 쿼리를 임베딩

search_results = search_faiss_index(query_embedding, faiss_index, df_embeddings)

# 결과 출력
for rank, (doc, distance) in enumerate(search_results):
    print(f"Rank {rank+1}: {doc} (Score: {distance})")



Rank 1: 제안서
페이지
제안서내용
수용
여부
요구사항명
요구사항
번호
77,
84
IV.1.2 보안관리방안,
V.1.1 품질보증개요
O
기술표준적용및운영체제호환성
준수
COR-001
34
66
III.2.3 DB 품질관리계획
O
표준지침준수
COR-002
35
67
III.3.1 인터페이스구현개요
O
웹표준, 웹접근성및호환성준수
COR-003
36
76 ~ 77
Ⅳ.1.2 보안관리방안
O
웹취약점점검및조치
COR-004
37
75
Ⅳ.1.1 사업관리방법론
O
사업수행일반사항준수및사업수행
조직구성
PMR-001
38
76 ~ 77
Ⅳ.1.2 보안관리방안
O
관리적보안대책
PMR-002
39
76 ~ 77
Ⅳ.1.2 보안관리방안
O
보안관련책임준수
PMR-003
40
79
Ⅳ.1.4 문서(형상)관리방안
O
사업수행계획서작성
PMR-004
41
78
Ⅳ.1.3 일정관리방안
O
일정관리및보고회개최
PMR-005
42
79
Ⅳ.1.4 문서(형상)관리방안
O
산출물관리
PMR-006
43
80
Ⅳ.1.5 위험관리방안
O
리스크관리
PMR-007
44
74
Ⅲ.4.5 이행및승인테스트수행방안
O
검사및검수
PMR-008
45
90 ~ 92
Ⅴ.3.1 교육훈련개요,
Ⅴ.3.2 교육훈련대상및방법,
Ⅴ.3.3 교육일정및절차
O
교육훈련
PSR-001
46
93 ~ 94
Ⅴ.4.1 기술이전개요,
Ⅴ.4.2 기술이전방법
O
기술이전
PSR-002
47
98
Ⅴ.6.1 하자보수방안
O
하자보수
PSR-003
48
95 ~ 96
Ⅴ.5.1.1 비상대책개요,
V.5.1.2 백업및복구방안
O
장애관리및백업
PSR-004
49
-
수용
O
EA 현행화
PSR-005
50
-
수용
O
소프트웨어사업정보제출
PSR-006
51 (Score: 134.1038818359375)
Rank 2: 제안서
페이지
제안서내용
수용
여부
요구사항명
요구사항
번호
61 ~ 62
III.1.1 장비성능품질보장방안
O
성능일반
PER-001
19
63
III.1.2 동시사용자접속수시험방안
O
동시사용자접속수
